Revision for GEOCO2AB

In [3]:

import xarray
import pandas as pd
import numpy as np

In [ ]:
#Mendistribusikan data GECO2AB to 17 crops including pastureland
#ini logika baru pasca email hermen

import xarray
import pandas as pd

GECO2AB = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/GECO2AB.NC", engine="netcdf4")
gfrac = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_5_mins_combined_run2.NC", engine="netcdf4")
garea = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GAREA.NC", engine="netcdf4")
gfrac_ngfbfc = [element.strip() for element in gfrac.coords['NGFBFC'].data.astype('str').tolist()]

garea_7020 = garea.isel(time=slice(0, 11))
garea_ha = np.nan_to_num(np.multiply(garea_7020['GAREA'].to_numpy(), 100))

gfrac_7020 = gfrac.isel(time=slice(0, 11)) 

gfrac_area = np.zeros((11, 17, 2160, 4320), dtype="float32")
for n in range (11):
    for i in range(17):
        gfrac_area[n][i] = np.nan_to_num(np.multiply(gfrac_7020['GFRAC_combined'].isel(time=n, NGFBFC=i), garea_ha[n]))

gfrac_area_netcdf = xarray.Dataset({
    "GFRAC_combined":(["time","NGFBFC","latitude", "longitude"], gfrac_area)
},coords={
        "time": pd.date_range(start='1970-01-01', end='2020-01-01', freq='5YS'),
        "NGFBFC" : gfrac_ngfbfc,
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

gfrac_area_netcdf_atribusi = gfrac_area_netcdf.isel(time=slice(1, 11))

#1975-2020
def divide(arr1, arr2):
    return np.divide(arr1, arr2)

def multiple(arr1, arr2):
    return np.multiply(arr1, arr2)

prop_crops_GFRAC = np.zeros((17, 10, 2160, 4320), dtype="float32")
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(10):
        prop_crops_GFRAC[i][n] = np.nan_to_num(divide(gfrac_area_netcdf_atribusi['GFRAC_combined'].isel(time=n).sel(NGFBFC=crop).to_numpy(), 
                                                            gfrac_area_netcdf_atribusi['GFRAC_combined'].isel(time=n).sum(dim='NGFBFC').to_numpy()))
        
#Referensi
Mg_to_tonne = 1
Kg_to_tonne = 1e-3
km2_to_ha = 1e2
ton_to_gigaton = 1e-9
#Konversi ton C ke ton CO2
#conversion rate C to CO2 = 44/12
#conversion rate N to N2O = 44/28
#conversion rate CH4 to CH4 = 1

#INPUT Unit GECO2AB Mg C/km2 yr
GECO2AB = GECO2AB.isel(time=slice(1, 11))
garea_7520 = garea.isel(time=slice(1, 11))
garea_garea_7520_ha = np.nan_to_num(np.multiply(garea_7520['GAREA'].to_numpy(), 100))


#Example conversion unit: 20.000 Mg C/km2 x (1 tonne/ 1 Mg)  / (100 ha/ 1km2) --> result in ton C/ha
emission_GECO2AB_ton_ha_C = GECO2AB['GECO2AB'] * Mg_to_tonne / km2_to_ha
#density to total per grid (tonC/ha * ha) --> result tonC/year
emission_ton_C = np.nan_to_num(np.multiply(emission_GECO2AB_ton_ha_C, garea_garea_7520_ha))

#CH4
ratioCH4toC = 0.00563 #(tonne/tonne)
emission_ton_CH4 = emission_ton_C * ratioCH4toC
#ton to Gton & x 5 tahun agar sama spt LUC, nanti dibagi 5 belakangan saat visualisasi
emission_GtTon_CH4 = emission_ton_CH4 * (1) * ton_to_gigaton * 5

emission_GtTon_CH4_netcdf = xarray.Dataset({
    "GECO2AB":(["time", "latitude", "longitude"], emission_GtTon_CH4)
},coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

#OUPUT Unit GECO2AB Gt CH4 (5 year)
GECO2AB_crop = np.zeros((17, 10, 2160, 4320), dtype="float32")
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(10):
        GECO2AB_crop[i][n] = np.nan_to_num(multiple(emission_GtTon_CH4_netcdf['GECO2AB'].isel(time=n).to_numpy(), prop_crops_GFRAC[i][n]))

GECO2AB_crop_CH4_netcdf = xarray.Dataset(
coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, GECO2AB_crop[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
GECO2AB_crop_CH4_netcdf = GECO2AB_crop_CH4_netcdf.assign(data_vars)

GECO2AB_crop_CH4_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/percrops/GECO2AB_crop_CH4.NC", mode="w", format="NETCDF4")

#N2O
ratioN2OtoC = 0.00009 #(tonne/tonne)
emission_ton_N2O = emission_ton_C * ratioN2OtoC
#ton to Gton & x 5 tahun agar sama spt LUC, nanti dibagi 5 belakangan saat visualisasi
emission_GtTon_NO2 = emission_ton_N2O * (44/28) * ton_to_gigaton * 5

emission_GtTon_NO2_netcdf = xarray.Dataset({
    "GECO2AB":(["time", "latitude", "longitude"], emission_GtTon_NO2)
},coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

#OUPUT Unit GECO2AB Gt N2O (5 year)
GECO2AB_crop = np.zeros((17, 10, 2160, 4320), dtype="float32")
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(10):
        GECO2AB_crop[i][n] = np.nan_to_num(multiple(emission_GtTon_NO2_netcdf['GECO2AB'].isel(time=n).to_numpy(), prop_crops_GFRAC[i][n]))

emission_GtTon_NO2_netcdf = xarray.Dataset(
coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, GECO2AB_crop[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
emission_GtTon_NO2_netcdf = emission_GtTon_NO2_netcdf.assign(data_vars)

emission_GtTon_NO2_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/percrops/GECO2AB_crop_N2O.NC", mode="w", format="NETCDF4")

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr

/home/cadlan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [13]:
GABOVERES = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/agri_burning/GABOVERES.NC", engine='h5netcdf')
AGWBUR = pd.read_excel("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/agri_burning/AGWBUR.xlsm")
GREG = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/agri_burning/GREG.nc", engine='h5netcdf')

In [10]:
gfrac_ngfbfc_ori = GABOVERES.coords['NGFBFC'].data.tolist()

gfrac_ngfbfc_ir_rf = []
gfrac_ngfbfc_rf = []
gfrac_ngfbfc_ir = []

for i, gfrac_cls in enumerate(gfrac_ngfbfc_ori):
    if np.char.replace(gfrac_cls,b' ','') == b"grass":
        gfrac_ngfbfc_ir.append(gfrac_cls)
        gfrac_ngfbfc_rf.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls)
    elif gfrac_cls[0:2] == b'RF':
        gfrac_ngfbfc_rf.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls[3:])
    elif gfrac_cls[0:2] == b'IR':
        gfrac_ngfbfc_ir.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls[3:])

gfrac_ngfbfc_ir_rf_list = gfrac_ngfbfc_ir_rf[:17]

In [7]:
gfrac_ngfbfc_ori = GABOVERES.coords['NGFBFC'].data.tolist()

gfrac_ngfbfc_ir_rf = []
gfrac_ngfbfc_rf = []
gfrac_ngfbfc_ir = []

for i, gfrac_cls in enumerate(gfrac_ngfbfc_ori):
    if np.char.replace(gfrac_cls,b' ','') == b"grass":
        gfrac_ngfbfc_ir.append(gfrac_cls)
        gfrac_ngfbfc_rf.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls)
    elif gfrac_cls[0:2] == b'RF':
        gfrac_ngfbfc_rf.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls[3:])
    elif gfrac_cls[0:2] == b'IR':
        gfrac_ngfbfc_ir.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls[3:])

gfrac_ngfbfc_ir_rf_list = gfrac_ngfbfc_ir_rf[:17]

GABOVERES_new = np.zeros((11, len(gfrac_ngfbfc_ir_rf_list), 2160, 4320), dtype='float')
for n in range(11):
    GABOVERES_new[n][0] = GABOVERES["GABOVERES"].isel(time=n, NGFBFC=0)
    GABOVERES_new[n][1] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=1) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=22))
    GABOVERES_new[n][2] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=2) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=23))
    GABOVERES_new[n][3] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=3) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=24))
    GABOVERES_new[n][4] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=4) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=25))
    GABOVERES_new[n][5] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=5) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=26))
    GABOVERES_new[n][6] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=6) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=27))
    GABOVERES_new[n][7] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=7) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=28))
    GABOVERES_new[n][8] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=8) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=29))
    GABOVERES_new[n][9] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=9) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=30))
    GABOVERES_new[n][10] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=10) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=31))
    GABOVERES_new[n][11] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=11) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=32))
    GABOVERES_new[n][12] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=12) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=33))
    GABOVERES_new[n][13] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=13) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=34))
    GABOVERES_new[n][14] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=14) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=35))
    GABOVERES_new[n][15] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=15) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=36))
    GABOVERES_new[n][16] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=16) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=37))

In [9]:
GABOVERES_netcdf = xr.Dataset({
    "GABOVERES_combined":(["time", "NGFBFC", "latitude", "longitude"], GABOVERES_new)
},coords={
    "time": pd.date_range(start="1970-01-01", end="2020-01-01", freq='5YS'),
    "NGFBFC" : gfrac_ngfbfc_ir_rf_list,
    "latitude": GABOVERES.coords["latitude"].to_numpy(),
    "longitude": GABOVERES.coords["longitude"].to_numpy()
})
GABOVERES_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/agri_burning/GABOVERES_combined.NC", mode="w", format="NETCDF4")

In [4]:
GABOVERES_netcdf = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/agri_burning/GABOVERES_combined.NC", engine="netcdf4")
GABOVERES_netcdf

<xarray.Dataset> Size: 14GB
Dimensions:             (time: 11, NGFBFC: 17, latitude: 2160, longitude: 4320)
Coordinates:
  * time                (time) datetime64[ns] 88B 1970-01-01 ... 2020-01-01
  * NGFBFC              (NGFBFC) |S50 850B b'grass                           ...
  * latitude            (latitude) float64 17kB 89.96 89.88 ... -89.87 -89.96
  * longitude           (longitude) float64 35kB -180.0 -179.9 ... 179.9 180.0
Data variables:
    GABOVERES_combined  (time, NGFBFC, latitude, longitude) float64 14GB ...

In [25]:
GABOVERES = GABOVERES_netcdf['GABOVERES_combined'].isel(NGFBFC=1).isel(time=2)

positive_values = GABOVERES.where(GABOVERES > 0, drop=False)
data_values = positive_values.values
indices = np.where(data_values > 0)
lat_indices = indices[0]
lon_indices = indices[1]
values = data_values[lat_indices, lon_indices]

result_list_GABOVERES_y = []

for lat_idx, lon_idx, val in zip(lat_indices, lon_indices, values):
    result_list_GABOVERES_y.append([lat_idx, lon_idx, val])

result_list_GABOVERES_y

[[228, 2446, 13.296427726745605],
 [228, 2492, 33.309967041015625],
 [229, 2487, 10.468756675720215],
 [229, 2510, 9.2850923538208],
 [230, 2453, 76.42627716064453],
 [230, 2462, 68.79210662841797],
 [231, 2436, 107.9911117553711],
 [232, 2436, 132.23965454101562],
 [233, 2425, 20.443891525268555],
 [233, 2427, 104.2625732421875],
 [233, 2433, 72.24085235595703],
 [233, 2434, 21.672761917114258],
 [233, 2460, 40.18410873413086],
 [233, 2477, 44.7028923034668],
 [234, 2443, 37.80126953125],
 [234, 2445, 104.50422668457031],
 [234, 2452, 111.7295150756836],
 [234, 2453, 196.1068572998047],
 [234, 2459, 39.62229919433594],
 [234, 2478, 29.276643753051758],
 [234, 2498, 2.5135374069213867],
 [235, 2472, 30.59429359436035],
 [236, 2430, 87.77310943603516],
 [236, 2431, 53.938350677490234],
 [236, 2467, 35.43201446533203],
 [236, 2476, 54.2103271484375],
 [236, 2497, 1.2651855945587158],
 [237, 2405, 29.555877685546875],
 [237, 2440, 76.18582916259766],
 [238, 2510, 36.33835983276367],
 [238

In [5]:
Mg_to_tonne = 1
Kg_to_tonne = 1e-3
km2_to_ha = 1e2
ton_to_gigaton = 1e-9

In [6]:
#Step 1
burnresLocc = np.zeros((11, 17, 2160, 4320), dtype='float32')

In [14]:
for n, year in enumerate(range(1970, 2020, 5)):
    for i, crops in enumerate(gfrac_ngfbfc_ir_rf_list):
        for j in range(1, 27):
            mask = np.where(GREG['GREG'].isel(time=0).to_numpy() == j)
            burnresLocc[n][i][mask] = GABOVERES_netcdf['GABOVERES_combined'].isel(time=n, NGFBFC=i).to_numpy()[mask] * AGWBUR[AGWBUR['year'] == year][j].values

In [16]:
burnresLocc_netcdf = xarray.Dataset({
    "burnresLocc":(["time", "NGFBFC", "latitude", "longitude"], burnresLocc)
},coords={
    "time": pd.date_range(start="1970-01-01", end="2020-01-01", freq='5YS'),
    "NGFBFC" : gfrac_ngfbfc_ir_rf_list,
    "latitude": GABOVERES.coords["latitude"].to_numpy(),
    "longitude": GABOVERES.coords["longitude"].to_numpy()
})

burnresLocc_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/agri_burning/burnresLocc.NC", mode="w", format="NETCDF4")

In [29]:
burnresLocc_netcdf.isel(time=2).isel(latitude=228, longitude=2446).isel(NGFBFC=1).to_pandas()

burnresLocc    0.370744
dtype: float64

In [51]:
ngfbfc = [element.strip() for element in burnresLocc_netcdf.coords['NGFBFC'].data.astype('str').tolist()]
ngfbfc

['grass',
 'Wheat',
 'Rice',
 'Maize',
 'Tropical cereals',
 'Other temperate cereals',
 'Pulses',
 'Soybeans',
 'Temperate oil crops',
 'Tropical oil crops',
 'Temperate roots & tubers',
 'Tropical roots & tubers',
 'Sugar crops',
 'Oil & palm fruit',
 'Vegetables & fruits',
 'Other non-food & luxury & spices',
 'Plant based fibres']

In [55]:
#Step 2a  (old unit Mg DM/km --> new unit tonne C/ha)
CVKGDMKGC = 0.48 #convert DM to C
km2_to_ha = 1e2  #convert km to ha
Mg_to_tonne = 1 #convert Mg to tonne

garea = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GAREA.NC", engine="netcdf4")
garea_7520 = garea.isel(time=slice(1, 11))
garea_7520_ha = np.nan_to_num(np.multiply(garea_7520['GAREA'].to_numpy(), 100))

eco2abLoc_ton_C_ha = ((burnresLocc_netcdf['burnresLocc'].isel(time=slice(1, 11)) * CVKGDMKGC * Mg_to_tonne) / km2_to_ha)

# Step2b (new unit tonne C)
eco2abLoc_ton_C = np.zeros((10, 17, 2160, 4320), dtype="float32")
for n in range (10):
    for i in range(17):
        eco2abLoc_ton_C[n][i] = np.nan_to_num(np.multiply(eco2abLoc_ton_C_ha.isel(time=n, NGFBFC=i), garea_7520_ha[n]))

eco2abLoc_ton_C_netcdf = xarray.Dataset({
    "eco2abLoc_ton_C":(["time","NGFBFC","latitude", "longitude"], eco2abLoc_ton_C)
},coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "NGFBFC" : ngfbfc,
        "latitude": GABOVERES.coords["latitude"].to_numpy(),
        "longitude": GABOVERES.coords["longitude"].to_numpy()
    })

In [56]:
burnresLocc_netcdf.isel(time=2).isel(latitude=228, longitude=2446).isel(NGFBFC=1).to_pandas()

burnresLocc    0.370744
dtype: float64

In [64]:
((0.370744 * CVKGDMKGC * Mg_to_tonne) / km2_to_ha * garea_7520.isel(time=0, latitude=228, longitude=2446) * 100).to_pandas()

GAREA    0.398814
dtype: float64

In [65]:
eco2abLoc_ton_C_netcdf.isel(time=1).isel(latitude=228, longitude=2446).isel(NGFBFC=1).to_pandas()

eco2abLoc_ton_C    0.398815
dtype: float64

In [69]:
eco2abLoc_ton_C_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/agri_burning/eco2abLoc_ton_C.NC", mode="w", format="NETCDF4")

In [71]:
eco2abLoc_ton_C_netcdf

<xarray.Dataset> Size: 6GB
Dimensions:          (time: 10, NGFBFC: 17, latitude: 2160, longitude: 4320)
Coordinates:
  * time             (time) datetime64[ns] 80B 1975-01-01 ... 2020-01-01
  * NGFBFC           (NGFBFC) <U32 2kB 'grass' 'Wheat' ... 'Plant based fibres'
  * latitude         (latitude) float64 17kB 89.96 89.88 89.79 ... -89.87 -89.96
  * longitude        (longitude) float64 35kB -180.0 -179.9 ... 179.9 180.0
Data variables:
    eco2abLoc_ton_C  (time, NGFBFC, latitude, longitude) float32 6GB 0.0 ... 0.0

In [75]:
#Step 3a CH4
RCH4CAGW = 5.63 #unit g CH4/ kg C
ratioCH4toC = 0.00563 #(tonne CH4 /tonne C)
CCH4toCH4 = 1

geCH4ab = eco2abLoc_ton_C_netcdf['eco2abLoc_ton_C'].to_numpy() * ratioCH4toC * CCH4toCH4
geCH4ab_GtTon = (geCH4ab * ton_to_gigaton * 5)
geCH4ab_GtTon = geCH4ab_GtTon.reshape((17, 10, 2160, 4320))

In [78]:
geCH4ab_GtTon_netcdf = xarray.Dataset(
coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": GABOVERES.coords["latitude"].to_numpy(),
        "longitude": GABOVERES.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, geCH4ab_GtTon[i]) for i, ngfbc in enumerate(ngfbfc)
}
geCH4ab_GtTon_netcdf = geCH4ab_GtTon_netcdf.assign(data_vars)
geCH4ab_GtTon_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/percrops/geCH4ab_crop_CH4.NC", mode="w", format="NETCDF4")

In [80]:
#Step 3b N2O
RN2OCAGW = 0.09 #unit g N/ kg C
ratioN2OtoC = 0.00009 #(tonne/tonne)
CNtoN2O = 44/28

geN2Oab = eco2abLoc_ton_C_netcdf['eco2abLoc_ton_C'].to_numpy() * ratioN2OtoC * CNtoN2O
geN2Oab_GtTon = (geN2Oab * ton_to_gigaton * 5)
geN2Oab_GtTon = geN2Oab_GtTon.reshape((17, 10, 2160, 4320))

GgeN2Oab_GtTon_netcdf = xarray.Dataset(
coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": GABOVERES.coords["latitude"].to_numpy(),
        "longitude": GABOVERES.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, geN2Oab_GtTon[i]) for i, ngfbc in enumerate(ngfbfc)
}
GgeN2Oab_GtTon_netcdf = GgeN2Oab_GtTon_netcdf.assign(data_vars)
GgeN2Oab_GtTon_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/percrops/geN2Oab_crop_N2O.NC", mode="w", format="NETCDF4")

REVISI GEN2ORE

In [81]:
#Mendistribusikan data GEN2ORE to 17 crops including pastureland

GEN2ORE = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/GEN2ORE.NC", engine="netcdf4")
gfrac = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_5_mins_combined_run2.NC", engine="netcdf4")
garea = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GAREA.NC", engine="netcdf4")
gfrac_ngfbfc = [element.strip() for element in gfrac.coords['NGFBFC'].data.astype('str').tolist()]

garea_7020 = garea.isel(time=slice(0, 11))
garea_ha = np.nan_to_num(np.multiply(garea_7020['GAREA'].to_numpy(), 100))

# gfrac_7020 = gfrac.isel(time=slice(0, 11)) 
# gfrac_area = np.zeros((11, 17, 2160, 4320), dtype="float32")
# for n in range (11):
#     for i in range(17):
#         gfrac_area[n][i] = np.nan_to_num(np.multiply(gfrac_7020['GFRAC_combined'].isel(time=n, NGFBFC=i), garea_ha[n]))

# gfrac_area_netcdf = xarray.Dataset({
#     "GFRAC_combined":(["time","NGFBFC","latitude", "longitude"], gfrac_area)
# },coords={
#         "time": pd.date_range(start='1970-01-01', end='2020-01-01', freq='5YS'),
#         "NGFBFC" : gfrac_ngfbfc,
#         "latitude": gfrac.coords["latitude"].to_numpy(),
#         "longitude": gfrac.coords["longitude"].to_numpy()
#     })

# gfrac_area_netcdf_atribusi = gfrac_area_netcdf.isel(time=slice(1, 11))

#1975-2020
def divide(arr1, arr2):
    return np.divide(arr1, arr2)

def multiple(arr1, arr2):
    return np.multiply(arr1, arr2)

GABOVERES_netcdf = GABOVERES_netcdf.isel(time=slice(1, 11))

prop_crops_GABOVERES = np.zeros((17, 10, 2160, 4320), dtype="float32")
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(10):
        prop_crops_GABOVERES[i][n] = np.nan_to_num(divide(GABOVERES_netcdf['GABOVERES_combined'].isel(NGFBFC=i).isel(time=n).to_numpy(), 
                                                            GABOVERES_netcdf['GABOVERES_combined'].isel(time=n).sum(dim='NGFBFC').to_numpy()))
        
#Referensi
Mg_to_tonne = 1
Kg_to_tonne = 1e-3
km2_to_ha = 1e2
ton_to_gigaton = 1e-9
#Konversi ton C ke ton CO2
#conversion rate C to CO2 = 44/12
#conversion rate N to N2O = 44/28
#conversion rate CH4 to CH4 = 1

#INPUT Unit GEN2ORE kg N/km2 yr
GEN2ORE = GEN2ORE.isel(time=slice(1, 11))
garea_7520 = garea.isel(time=slice(1, 11))
garea_garea_7520_ha = np.nan_to_num(np.multiply(garea_7520['GAREA'].to_numpy(), 100))

#Example conversion unit: 20.000 Mg C/km2 x (1 tonne/ 1 Mg)  / (100 ha/ 1km2) --> result in ton C/ha
emission_GEN2ORE_ton_ha = GEN2ORE['GEN2ORE'] * Kg_to_tonne / km2_to_ha
#density to total per grid (tonC/ha * ha) --> result tonC/year
emission_ton = np.nan_to_num(np.multiply(emission_GEN2ORE_ton_ha, garea_garea_7520_ha))
#ton to Gton & x 5 tahun agar sama spt LUC, nanti dibagi 5 belakangan saat visualisasi
emission_GtTon = emission_ton * (44/28) * ton_to_gigaton * 5

emission_GtTon_netcdf = xarray.Dataset({
    "GEN2ORE":(["time", "latitude", "longitude"], emission_GtTon)
},coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

#OUPUT Unit GEN2ORE Gt Ton N2O (5 year)
GEN2ORE_crop = np.zeros((17, 10, 2160, 4320), dtype="float32")
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(10):
        GEN2ORE_crop[i][n] = np.nan_to_num(multiple(emission_GtTon_netcdf['GEN2ORE'].isel(time=n).to_numpy(), prop_crops_GABOVERES[i][n]))

GEN2ORE_crop_netcdf = xarray.Dataset(
coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, GEN2ORE_crop[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
GEN2ORE_crop_netcdf = GEN2ORE_crop_netcdf.assign(data_vars)



/tmp/ipykernel_3990511/3245781438.py:30: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
/tmp/ipykernel_3990511/3245781438.py:30: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
/tmp/ipykernel_3990511/3245781438.py:30: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
/tmp/ipykernel_3990511/3245781438.py:30: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
/tmp/ipykernel_3990511/3245781438.py:30: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
/tmp/ipykernel_3990511/3245781438.py:30: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
/tmp/ipykernel_3990511/3245781438.py:30: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
/tmp/ipykernel_3990511/3245781438.py:30: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
/tmp/ipy

In [82]:
GEN2ORE_crop_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/grid_emissions/percrops/GEN2ORE_crop_v2.NC", mode="w", format="NETCDF4")